In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -rm -r -f /tmp/output

In [3]:
!hdfs dfs -mkdir /tmp/output

In [4]:
!hdfs dfs -copyFromLocal data.tsv /tmp/output/

In [6]:
%%hive

DROP TABLE IF EXISTS datos;

CREATE TABLE datos (letra       STRING,
                   fecha       STRING,
                   numero      INT
                  )

ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
LOAD DATA INPATH '/tmp/output/data.tsv' OVERWRITE
INTO TABLE datos;

DROP TABLE IF EXISTS datos;
OK
Time taken: 0.576 seconds
CREATE TABLE datos (letra       STRING,
                   fecha       STRING,
                   numero      INT
                  )
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
OK
Time taken: 0.065 seconds
LOAD DATA INPATH '/tmp/output/data.tsv' OVERWRITE
INTO TABLE datos;
Loading data to table default.datos
OK
Time taken: 1.161 seconds


In [12]:
%%hive

DROP TABLE IF EXISTS resultado;

CREATE TABLE resultado
AS
SELECT  letra,
        COUNT (letra)
FROM 
    datos
GROUP BY 
    letra
;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM resultado;

DROP TABLE IF EXISTS resultado;
OK
Time taken: 0.006 seconds
CREATE TABLE resultado
AS
SELECT  letra,
        COUNT (letra)
FROM 
    datos
GROUP BY 
    letra
;
Query ID = root_20200119142831_ba0f752f-3484-4adf-b7e8-eecf2e8ef8dc
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579442391358_0001, Tracking URL = http://473fcc274f09:8088/proxy/application_1579442391358_0001/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579442391358_0001
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-19 14:28:44,491 Stage-1 map = 0%,  reduce = 0%
2020-01-19 14:28:49,990 Stage-1 map = 100%

In [13]:
%%hive
SELECT * FROM resultado LIMIT 5;

SELECT * FROM resultado LIMIT 5;
OK
A	12
B	10
C	4
D	1
E	13
Time taken: 0.274 seconds, Fetched: 5 row(s)


In [14]:
!hdfs dfs -copyToLocal /tmp/output output;

In [15]:
%%hive
DROP TABLE resultado;

DROP TABLE resultado;
OK
Time taken: 0.079 seconds


In [16]:
%hive_quit